<a href="https://colab.research.google.com/github/hdemma/hdemma.github.io/blob/master/Macro_Prediction_Models/addingElevation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import rasterio
from pyproj import Proj, transform
import csv
import pandas as pd
import tqdm

In [ ]:
Vehicle_Type = 'Electric_Vehicles'
Vehicle_ID = [751]

Filesize of w001001.adf is too big

In [ ]:
## Must be same format as test_coordinates, Longitude, Latitude,
## with one row for header
# elevation = '../DataSet/Elevation/w001001.adf'
elevation = 'data/w001001.adf'

def GettingElevation(xs,ys):
    Elevation = []
    with rasterio.open(elevation) as src:
        if(len(xs)==len(ys)):
            for i in tqdm.tqdm(range (len(xs))):
                ## First, need to project google coordinates
                ## into the same coordinate system as the raster
                inProj = Proj('epsg:4326')
                outProj = Proj(src.crs)
                x1, y1 = xs[i], ys[i]
                x2,y2 = transform(inProj,outProj,y1,x1)
                for val in src.sample([(x2, y2)]):
                    Elevation.append(val[0])
        else:
            print("Uneven arrays. Try again")


    return Elevation

In [ ]:
for v in Vehicle_ID:
    Vehicle_Name = f'BYD_{v}'
    print(f'Processing {Vehicle_Name}')

    df = pd.read_csv(f'{Vehicle_Name}_with_all_Samples_without_Outliers.csv',low_memory=False)
    print(len(df))

    Initial_Lat = list(df['Initial_recorded_Latitude'])
    Initial_Long = list(df['Initial_recorded_Longitude'])
    Final_Lat = list(df['Final_recorded_Latitude'])
    Final_Long = list(df['Final_recorded_Longitude'])


    print('Start_Elevation\n')
    Start_Elevation = GettingElevation(Initial_Long,Initial_Lat)
    print('Stop_Elevation\n')
    Stop_Elevation = GettingElevation(Final_Long,Final_Lat)

    Change_In_Elevation = []
    for i in range(len(Start_Elevation)):
        diff = Stop_Elevation[i] - Start_Elevation[i]
        Change_In_Elevation.append(diff)

    
    df['Change_In_Elevation'] = pd.Series(Change_In_Elevation)
    csv = (f'{Vehicle_Name}_With_Elevation_Data.csv')
    df.to_csv(csv, index=False)